In [ ]:
import torch
from maps import model_map
from transformers import pipeline
from prompt_template import REASONING_JUDGE

model_str = 'llama_3_1_8B_it'
model_dir = model_map[model_str]
model = pipeline("text-generation", 
                model=model_dir,
                torch_dtype=torch.float16,
                max_length=8024,
                device=0)

model

In [16]:
target_model_str = 'deepseek_14b'
target_model_dir = model_map[target_model_str]
target_model = pipeline("text-generation", 
                model=target_model_dir,
                torch_dtype=torch.float16,
                max_length=8024,
                device=0)

target_model

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
import pandas as pd
from datasets import load_dataset

ds = load_dataset("greengerong/leetcode")['train']
df = pd.DataFrame.from_dict(ds)
df.head()

,id,slug,title,difficulty,content,java,c++,python,javascript
0,1,two-sum,Two Sum,Easy,Given an array of integers `nums` and an integ...,\n ```java\nimport java.util.HashMap;\nimpo...,\n ```cpp\n#include <vector>\n#include <uno...,"\n ```python\ndef twoSum(nums, target):\n ...","\n ```javascript\nfunction twoSum(nums, tar..."
1,2,add-two-numbers,Add Two Numbers,Medium,You are given two **non-empty** linked lists r...,\n ```java\npublic class ListNode {\n in...,\n ```cpp\nstruct ListNode {\n int val;\...,\n ```python\nclass ListNode:\n def __in...,"\n ```javascript\nfunction ListNode(val, ne..."
2,3,longest-substring-without-repeating-characters,Longest Substring Without Repeating Characters,Medium,"Given a string `s`, find the length of the **l...",\n ```java\nimport java.util.HashSet;\nimpo...,\n ```cpp\n#include <string>\n#include <uno...,\n ```python\ndef length_of_longest_substri...,\n ```javascript\nfunction lengthOfLongestS...
3,4,median-of-two-sorted-arrays,Median of Two Sorted Arrays,Hard,Given two sorted arrays `nums1` and `nums2` of...,\n ```java\npublic double findMedianSortedA...,\n ```cpp\ndouble findMedianSortedArrays(ve...,\n ```python\ndef findMedianSortedArrays(nu...,\n ```javascript\nfunction findMedianSorted...
4,5,longest-palindromic-substring,Longest Palindromic Substring,Medium,"Given a string `s`, return _the longest_ _pali...",\n ```java\npublic String longestPalindromi...,\n ```cpp\n#include <string>\n\nstd::string...,\n ```python\ndef longest_palindromic_subst...,\n ```javascript\nfunction longestPalindrom...


In [5]:
idx = 1
orig_content = df['content'][idx]
orig_content

'You are given two **non-empty** linked lists representing two non-negative integers. The digits are stored in **reverse order**, and each of their nodes contains a single digit. Add the two numbers and return the sum as a linked list.\n\nYou may assume the two numbers do not contain any leading zero, except the number 0 itself.\n\n**Example 1:**\n\n**Input:** l1 = \\[2,4,3\\], l2 = \\[5,6,4\\]\n**Output:** \\[7,0,8\\]\n**Explanation:** 342 + 465 = 807.\n\n**Example 2:**\n\n**Input:** l1 = \\[0\\], l2 = \\[0\\]\n**Output:** \\[0\\]\n\n**Example 3:**\n\n**Input:** l1 = \\[9,9,9,9,9,9,9\\], l2 = \\[9,9,9,9\\]\n**Output:** \\[8,9,9,9,0,0,0,1\\]\n\n**Constraints:**\n\n*   The number of nodes in each linked list is in the range `[1, 100]`.\n*   `0 <= Node.val <= 9`\n*   It is guaranteed that the list represents a number that does not have leading zeros.'

### Prompt Templates

In [6]:
STORYTELLING_TEMPLATE = """
Please convert the following coding problem into a storytelling framework while preserving all the original problem's constraints and requirements. The story should:

1. Be written as a fictional narrative where characters encounter the problem naturally
2. Maintain all mathematical/logical constraints from the original
3. Include all examples from the original in narrative form
4. Keep the same variable names and technical details
5. Present the problem implicitly rather than explicitly
6. Have a natural flow that would make sense in a story context

Here is the original problem:
{orig_content}

Create the story version now, maintaining all technical accuracy while making it engaging. Only generate the output with no additional comment:
"""

In [7]:
NUMEMRICAL_PERTURBATION_TEMPLATE = f"""
Please modify the numerical values in the following coding problem while maintaining:
1. All structural/logical relationships between values
2. The original problem's constraints (adjusted proportionally)
3. The same number of examples
4. The same variable names and problem framework
5. Mathematical consistency

Apply one or more of these transformations to each number:
- Random increment/decrement (small-medium change)
- Proportional scaling (e.g., multiply all by 1.5)
- Sign flipping (if mathematically valid)
- Digit permutation (e.g., 15 → 51)
- Base number change (e.g., shift from 9 to 16 as base)

Original problem:
{orig_content}

Provide 3 different variations, each with:
- A description of the transformation applied
- The complete modified problem
- Verification that all constraints remain logically consistent
"""

In [ ]:
domain_shift_prompt = """
Rephrase this coding problem in a different domain (e.g., medicine, sports, physics) while:
1. Keeping all mathematical/logical requirements intact.
2. Using domain-specific terminology for variables (e.g., "target" → "patient’s threshold").
3. Ensuring the solution remains identical to the original.

Original problem:
{orig_content}

Generate 2 versions: one in a scientific domain and one in an everyday scenario.
"""

In [ ]:
redundancy_prompt = f"""
Modify this problem to include plausible but irrelevant computational steps:
1. Insert a preprocessing requirement (e.g., "sort the array first").
2. Add a false intermediate goal (e.g., "compute the average before solving").
3. Ensure the core problem is still solvable but obscured.

Original problem:
{orig_content}

Output the modified version with injected redundancy.
"""

In [11]:
antagonistic_example_prompt = f"""
Add 2-3 adversarial examples that:
1. Follow a false pattern (e.g., always select even indices).
2. Include plausible-sounding but incorrect explanations.
3. Are statistically likely to appear in naive sampling.

Original: {orig_content}

Generate bait examples and their misleading logic.
"""

### Prompt Conversion

In [15]:
prompt = NUMEMRICAL_PERTURBATION_TEMPLATE.format(orig_content=orig_content)
print(prompt)


Please modify the numerical values in the following coding problem while maintaining:
1. All structural/logical relationships between values
2. The original problem's constraints (adjusted proportionally)
3. The same number of examples
4. The same variable names and problem framework
5. Mathematical consistency

Apply one or more of these transformations to each number:
- Random increment/decrement (small-medium change)
- Proportional scaling (e.g., multiply all by 1.5)
- Sign flipping (if mathematically valid)
- Digit permutation (e.g., 15 → 51)
- Base number change (e.g., shift from 9 to 16 as base)

Original problem:
You are given two **non-empty** linked lists representing two non-negative integers. The digits are stored in **reverse order**, and each of their nodes contains a single digit. Add the two numbers and return the sum as a linked list.

You may assume the two numbers do not contain any leading zero, except the number 0 itself.

**Example 1:**

**Input:** l1 = \[2,4,3\],

In [17]:
messages = [{"role": "user", "content": prompt}]

modified_content = model(messages)
modified_content = modified_content[0]['generated_text'][1]['content']
modified_content

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'### Transformation 1: Random Increment/Decrement and Proportional Scaling\n\n**Description:** \nWe will apply a small random increment/decrement to some of the numerical values and scale all the values proportionally by a factor of 2.\n\n**Modified Problem:**\nYou are given two **non-empty** linked lists representing two non-negative integers. The digits are stored in **reverse order**, and each of their nodes contains a single digit. Add the two numbers and return the sum as a linked list.\n\nYou may assume the two numbers do not contain any leading zero, except the number 0 itself.\n\n**Example 1:**\n\n**Input:** l1 = \\[5,8,3\\], l2 = \\[7,6,4\\]\n**Output:** \\[2,0,8\\]\n**Explanation:** 538 + 764 = 1302.\n\n**Example 2:**\n\n**Input:** l1 = \\[0\\], l2 = \\[0\\]\n**Output:** \\[0\\]\n\n**Example 3:**\n\n**Input:** l1 = \\[19,9,9,9,9,9,9\\], l2 = \\[9,9,9,19\\]\n**Output:** \\[9,9,9,8,0,0,0,1\\]\n\n**Constraints:**\n\n*   The number of nodes in each linked list is in the range `[1